In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

26 documents
6 classes ['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']
48 unique lemmatized words ['a', 'age', 'am', 'anyone', 'are', 'buy', 'call', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodbye', 'guy', 'have', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'later', 'leaving', 'like', 'menu', 'name', 'of', 'old', 'on', 'open', 'operation', 'reccommend', 'see', 'should', 'something', 'the', 'there', 'tim', 'to', 'up', 'what', 'whats', 'when', 'you', 'your']


In [5]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
26/26 [==============================] - 4s 144ms/step - loss: 1.7446 - accuracy: 0.1538
Epoch 2/200
26/26 [==============================] - 0s 1ms/step - loss: 1.8664 - accuracy: 0.0769
Epoch 3/200
26/26 [==============================] - 0s 2ms/step - loss: 1.7398 - accuracy: 0.2692
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 1.6268 - accuracy: 0.2692
Epoch 5/200
26/26 [==============================] - 0s 2ms/step - loss: 1.6336 - accuracy: 0.3846
Epoch 6/200
26/26 [==============================] - 0s 1ms/step - loss: 1.5740 - accuracy: 0.4615
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 1.4530 - accuracy: 0.4615
Epoch 8/200
26/26 [==============================] - 0s 1ms/step - loss: 1.4169 - accuracy: 0.5000
Epoch 9/200
26/26 [==============================] - 0s 1ms/step - loss: 1.4904 - accuracy: 0.4231
Epoch 10/200
26/26 [==============================] - 0s 1ms/step - loss: 1.4092 - accuracy: 0.4231
Epoch 1

26/26 [==============================] - 0s 1ms/step - loss: 0.0310 - accuracy: 1.0000
Epoch 83/200
26/26 [==============================] - 0s 601us/step - loss: 0.0700 - accuracy: 1.0000
Epoch 84/200
26/26 [==============================] - 0s 601us/step - loss: 0.0591 - accuracy: 1.0000
Epoch 85/200
26/26 [==============================] - 0s 1ms/step - loss: 0.1000 - accuracy: 0.9615
Epoch 86/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0279 - accuracy: 1.0000
Epoch 87/200
26/26 [==============================] - 0s 1ms/step - loss: 0.1013 - accuracy: 0.9615
Epoch 88/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0185 - accuracy: 1.0000
Epoch 89/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0489 - accuracy: 0.9615
Epoch 90/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0673 - accuracy: 1.0000
Epoch 91/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0623 - accuracy: 0.9615
Epoch 92/

26/26 [==============================] - 0s 2ms/step - loss: 0.0101 - accuracy: 1.0000
Epoch 163/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0380 - accuracy: 1.0000
Epoch 164/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 165/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0262 - accuracy: 1.0000
Epoch 166/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0339 - accuracy: 1.0000
Epoch 167/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0797 - accuracy: 0.9615
Epoch 168/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0973 - accuracy: 0.9615
Epoch 169/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0605 - accuracy: 0.9615
Epoch 170/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 171/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0211 - accuracy: 1.0000
Epoc

In [7]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()